# Import the required modules.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Number_of_samples determine how many samples from the attack and normal dataset should be read and used.

In [ ]:
number_of_samples = 50000

# Read data from attack and normal datasets.

In [ ]:
data_attack = pd.read_csv('/content/dataset_attack.csv', nrows=number_of_samples)

In [ ]:
data_normal = pd.read_csv('/content/dataset_normal.csv', nrows=number_of_samples)

In [ ]:

data_normal.columns = ['frame.len', 'frame.protocols', 'ip.hdr_len',
                       'ip.len', 'ip.flags.rb', 'ip.flags.df', 'p.flags.mf', 'ip.frag_offset',
                       'ip.ttl', 'ip.proto', 'ip.src', 'ip.dst', 'tcp.srcport', 'tcp.dstport',
                       'tcp.len', 'tcp.ack', 'tcp.flags.res', 'tcp.flags.ns', 'tcp.flags.cwr',
                       'tcp.flags.ecn', 'tcp.flags.urg', 'tcp.flags.ack', 'tcp.flags.push',
                       'tcp.flags.reset', 'tcp.flags.syn', 'tcp.flags.fin', 'tcp.window_size',
                       'tcp.time_delta', 'class']
data_attack.columns = ['frame.len', 'frame.protocols', 'ip.hdr_len',
                       'ip.len', 'ip.flags.rb', 'ip.flags.df', 'p.flags.mf', 'ip.frag_offset',
                       'ip.ttl', 'ip.proto', 'ip.src', 'ip.dst', 'tcp.srcport', 'tcp.dstport',
                       'tcp.len', 'tcp.ack', 'tcp.flags.res', 'tcp.flags.ns', 'tcp.flags.cwr',
                       'tcp.flags.ecn', 'tcp.flags.urg', 'tcp.flags.ack', 'tcp.flags.push',
                       'tcp.flags.reset', 'tcp.flags.syn', 'tcp.flags.fin', 'tcp.window_size',
                       'tcp.time_delta', 'class']

# Drop unwanted columns

In [ ]:
data_normal = data_normal.drop(['ip.src', 'ip.dst', 'frame.protocols'], axis=1)
data_attack = data_attack.drop(['ip.src', 'ip.dst', 'frame.protocols'], axis=1)

In [ ]:

features = ['frame.len', 'ip.hdr_len', 'ip.len', 'ip.flags.rb', 'ip.flags.df', 'p.flags.mf', 'ip.frag_offset',
            'ip.ttl', 'ip.proto', 'tcp.srcport', 'tcp.dstport', 'tcp.len', 'tcp.ack', 'tcp.flags.res',
            'tcp.flags.ns', 'tcp.flags.cwr', 'tcp.flags.ecn', 'tcp.flags.urg', 'tcp.flags.ack',
            'tcp.flags.push', 'tcp.flags.reset', 'tcp.flags.syn', 'tcp.flags.fin', 'tcp.window_size',
            'tcp.time_delta']


In [ ]:
X_normal = data_normal[features].values
X_attack = data_attack[features].values
Y_normal = data_normal['class']
Y_attack = data_attack['class']
X = np.concatenate((X_normal, X_attack))
Y = np.concatenate((Y_normal, Y_attack))

# Standardise the data

In [ ]:
scalar = StandardScaler(copy=True, with_mean=True, with_std=True)
scalar.fit(X)
X = scalar.transform(X)
X[np.isnan(X)] = 0
X[np.isinf(X)] = 0

for i in range(0, len(Y)):
    if Y[i] == "attack":
        Y[i] = 0
    else:
        Y[i] = 1

le = LabelEncoder()
Y = le.fit_transform(Y)

# Transformation

In [ ]:
features = len(X[0])
samples = X.shape[0]
train_len = 25
input_len = samples - train_len
I = np.zeros((samples - train_len, train_len, features))

for i in range(input_len):
    temp = np.zeros((train_len, features))
    for j in range(i, i + train_len - 1):
        temp[j - i] = X[j]
    I[i] = temp

# Splitting the dataset into training and validation sets

In [ ]:
    
X_train, X_test, Y_train, Y_test = train_test_split(I, Y[train_len:100000], test_size=0.2)

In [ ]:
X.shape

# Three kinds of D4C

In [ ]:
class BiD4C(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BiD4C, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=32, kernel_size=3, stride=1)
        self.bn1 = nn.BatchNorm1d(32)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1)
        self.bn2 = nn.BatchNorm1d(64)
        self.rnn = nn.LSTM(input_size=64, hidden_size=hidden_size, bidirectional=True, batch_first=True)
        self.fc1 = nn.Linear(hidden_size * 2, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, output_size)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, out):
        out = out.permute(0, 2, 1)  # 将数据从 (batch_size, seq_len, input_size) 转换为 (batch_size, input_size, seq_len)
        out = self.conv1(out)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = out.permute(0, 2, 1)  # 将数据从 (batch_size, input_size, seq_len) 转换为 (batch_size, seq_len, hidden_size)
        out, _ = self.rnn(out)
        out = out[:, -1, :]
        out = self.fc1(out)
        out = self.bn3(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

## Model parameters

In [ ]:
input_size = features
hidden_size = 64
output_size = 1
learning_rate = 0.001
num_epochs = 40
batch_size = 128

# Set model, optimizer and loss function

In [ ]:
model = BiD4C(input_size, hidden_size, output_size)

# Load Data & Set Optimizer

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
train_dataset = TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(Y_train).float())
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
test_dataset = TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(Y_test).float())
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
train_losses = []
test_accuracies = []

# Train the model on the training set

In [ ]:
for epoch in range(num_epochs):
  running_loss = 0.0
  # use a variable to store the number of batches per validation
  val_freq = 125
  for i, data in enumerate(train_loader, 1): # start the index from 1
    inputs, labels = data
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs.squeeze(), labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()

    # check if the current batch is a multiple of val_freq
    if i % val_freq == 0:
      # perform validation
      with torch.no_grad():
        model.eval()
        correct = 0
        total = 0
        for data in test_loader:
          inputs, labels = data
          outputs = model(inputs)
          predicted = (outputs.squeeze() > 0.5).float()
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_losses.append(train_loss)

        
        test_accuracy = 100 * correct / total
        test_accuracies.append(test_accuracy)

        print('Epoch [{}/{}], Batch [{}/{}], Train Loss: {:.4f}, Test Acc: {:.2f}%'
        .format(epoch + 1, num_epochs, i, len(train_loader), running_loss / val_freq, test_accuracy))
      
      # reset the running loss
      running_loss = 0.0

      # switch back to train mode
      model.train()


# Validated on the validation set

In [ ]:
model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for data in test_loader:
        inputs, labels = data
        outputs = model(inputs)
        predicted = (outputs.squeeze() > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test data: {} %'.format(100 * correct / total))

# Draw the change graph of loss and correct rate during training

In [ ]:

import matplotlib.pyplot as plt
# Create a figure object with two subplots
fig, (ax1, ax2) = plt.subplots(2)
# Plot training loss values on the first subplot
ax1.plot(train_losses)
# Set title, labels and legend
ax1.set_title('D4C Recurrent Network Loss')
ax1.set_ylabel('Loss')
ax1.set_xlabel('1/5 Epoch')
ax1.legend(['Train Loss'], loc='upper right')
# Plot test accuracy values on the second subplot
ax2.plot(test_accuracies)
# Set title, labels and legend
ax2.set_title('D4C Recurrent Network Accuracy')
ax2.set_ylabel('Accuracy(%)')
ax2.set_xlabel('1/5 Epoch')
ax2.legend(['Test Accuracy'], loc='lower right')
# Adjust the layout of the figure
fig.tight_layout()
# Save the figure as an image file
plt.savefig('D4C Loss and Accuracy.png')
# Show the figure
plt.show()


# Compute and visualize the confusion matrix

In [ ]:
with torch.no_grad():
    model.eval()
    predict = model(torch.from_numpy(X_test).float()).numpy()

predictn = predict.flatten().round()
predictn = predictn.tolist()
Y_testn = Y_test.tolist()

tn, fp, fn, tp = confusion_matrix(Y_testn, predictn).ravel()
to_heat_map = [[tn, fp], [fn, tp]]
to_heat_map = pd.DataFrame(to_heat_map, index=["Attack", "Normal"], columns=["Attack", "Normal"])
ax = sns.heatmap(to_heat_map, annot=True, fmt="d")

figure = ax.get_figure()
figure.savefig('D4C Confusion Matrix.png', dpi=400)

# Save the model

In [ ]:
torch.save(model, 'D4C Model.pt')